In [1]:
from remi_z import MultiTrack
import torch

In [2]:
latent_fp = '/data1/longshen/Datasets/Piano/POP909/latents/song_level_phr_seq/all_songs_latents.pt'
latent = torch.load(latent_fp)
latent.shape

torch.Size([857, 784, 128])

In [3]:
train_data = torch.load('/data1/longshen/Datasets/Piano/POP909/latents/song_level_phr_seq_with_annot/train_data.pt')
valid_data = torch.load('/data1/longshen/Datasets/Piano/POP909/latents/song_level_phr_seq_with_annot/valid_data.pt')
print(len(train_data), len(valid_data))
print(train_data['001'])

734 81
{'latent': tensor([[-7.1627e-01,  9.7244e-01, -1.0044e+00,  ..., -8.0386e-03,
          4.6785e-01, -7.6724e-02],
        [-1.8810e+00,  1.0133e+00, -1.6126e-01,  ..., -5.4187e-02,
         -2.4041e-01, -1.2703e+00],
        [ 1.5169e-02, -1.3356e-04, -9.7072e-04,  ...,  1.9428e-03,
          1.1417e-02, -2.1385e-02],
        ...,
        [-1.8340e-01, -3.3638e-01, -9.0959e-01,  ...,  6.6031e-02,
          1.0439e+00, -1.2441e+00],
        [ 1.5169e-02, -1.3356e-04, -9.7074e-04,  ...,  1.9428e-03,
          1.1417e-02, -2.1385e-02],
        [ 1.5169e-02, -1.3356e-04, -9.7074e-04,  ...,  1.9428e-03,
          1.1417e-02, -2.1385e-02]]), 'length': 284, 'phrase_annotation': [('i', 4), ('A', 4), ('B', 8), ('A', 4), ('A', 4), ('b', 4), ('B', 8), ('A', 4), ('A', 4), ('b', 4), ('b', 4), ('A', 4), ('A', 4), ('b', 4), ('A', 4), ('o', 3)]}


In [4]:
import os
import sys
sys.path.append('/home/longshen/work/AccGen/AccGen')
from models.vae_inference import PhraseVAE
model = PhraseVAE()
decoded = model.decode_batch(train_data['001']['latent'].unsqueeze(0))

/home/longshen/programs/miniconda3/envs/accgen/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using compress_style: first_n_tokens
Adaptive scale: False


In [5]:
from remi_z import MultiTrack
from models.vae_inference import remove_extra_bar_line
t = remove_extra_bar_line(decoded[0])
mt = MultiTrack.from_remiz_str(t)
mt.set_tempo(90)
save_path = '/home/longshen/work/AccGen/test_outputs/dataset/decoded_001.mid'
mt.to_midi(save_path)

MIDI file successfully written to /home/longshen/work/AccGen/test_outputs/dataset/decoded_001.mid


In [6]:
from datasets.dataset_utils import calculate_bar_id_of_phrases
import random
# set seed for reproducibility
random.seed(42)

# C(n-2, k) combination
def combination_augment(latent, annot):
    bar_id_of_phrase = calculate_bar_id_of_phrases(annot)
    print(bar_id_of_phrase)
    body = bar_id_of_phrase[1:-1]
    n = len(body)
    # random select k from [1, n]
    k = random.randint(1, n)
    
    ids = list(range(n))
    selected_ids = random.sample(ids, k)
    selected_ids.sort()
    selected = [body[i] for i in selected_ids]
    selected = [bar_id_of_phrase[0]] + selected + [bar_id_of_phrase[-1]]
    print("Selected phrase bar ids:")
    print(selected)

    # Permute the latents according to selected phrase bar ids
    new_latent = []
    for i in range(len(selected)):
        start_bar = selected[i][1]
        end_bar = selected[i][2]
        start_idx = start_bar * 4
        end_idx = end_bar * 4
        print(f'start_bar: {start_bar}, end_bar: {end_bar}, start_idx: {start_idx}, end_idx: {end_idx}')
        new_latent.append(latent[start_idx:end_idx])
    new_latent = torch.cat(new_latent, dim=0)
    print("Original latent shape:", latent.shape)
    print("New latent shape:", new_latent.shape)
    return new_latent

latent_c = combination_augment(train_data['001']['latent'], train_data['001']['phrase_annotation'])
decoded_c = model.decode_batch(latent_c.unsqueeze(0))
t_c = remove_extra_bar_line(decoded_c[0])
mt_c = MultiTrack.from_remiz_str(t_c)
mt_c.set_tempo(90)
save_path_c = '/home/longshen/work/AccGen/test_outputs/dataset/decoded_001_c.mid'
mt_c.to_midi(save_path_c)

[('i', 0, 4), ('A', 4, 8), ('B', 8, 16), ('A', 16, 20), ('A', 20, 24), ('b', 24, 28), ('B', 28, 36), ('A', 36, 40), ('A', 40, 44), ('b', 44, 48), ('b', 48, 52), ('A', 52, 56), ('A', 56, 60), ('b', 60, 64), ('A', 64, 68), ('o', 68, 71)]
Selected phrase bar ids:
[('i', 0, 4), ('A', 4, 8), ('B', 8, 16), ('A', 16, 20), ('A', 20, 24), ('b', 24, 28), ('B', 28, 36), ('A', 36, 40), ('b', 48, 52), ('A', 52, 56), ('A', 56, 60), ('b', 60, 64), ('o', 68, 71)]
start_bar: 0, end_bar: 4, start_idx: 0, end_idx: 16
start_bar: 4, end_bar: 8, start_idx: 16, end_idx: 32
start_bar: 8, end_bar: 16, start_idx: 32, end_idx: 64
start_bar: 16, end_bar: 20, start_idx: 64, end_idx: 80
start_bar: 20, end_bar: 24, start_idx: 80, end_idx: 96
start_bar: 24, end_bar: 28, start_idx: 96, end_idx: 112
start_bar: 28, end_bar: 36, start_idx: 112, end_idx: 144
start_bar: 36, end_bar: 40, start_idx: 144, end_idx: 160
start_bar: 48, end_bar: 52, start_idx: 192, end_idx: 208
start_bar: 52, end_bar: 56, start_idx: 208, end_idx:

In [7]:
# Check if all song start with 'i' and end with 'o' phrase type
for key in train_data.keys():
    annot = train_data[key]['phrase_annotation']
    if annot[0][0] != 'i' or annot[-1][0] != 'o':
        print(f'Error in song {key}: start with {annot[0][0]}, end with {annot[-1][0]}')

Error in song 004: start with A, end with o
Error in song 005: start with i, end with X
Error in song 006: start with X, end with X
Error in song 007: start with i, end with X
Error in song 008: start with i, end with X
Error in song 009: start with A, end with X
Error in song 012: start with i, end with X
Error in song 016: start with i, end with X
Error in song 023: start with i, end with X
Error in song 027: start with i, end with X
Error in song 030: start with i, end with X
Error in song 040: start with i, end with X
Error in song 041: start with i, end with X
Error in song 046: start with A, end with o
Error in song 047: start with A, end with o
Error in song 048: start with i, end with X
Error in song 049: start with A, end with o
Error in song 060: start with i, end with X
Error in song 064: start with i, end with X
Error in song 065: start with i, end with X
Error in song 073: start with i, end with X
Error in song 077: start with i, end with C
Error in song 078: start with i,